In [ ]:
!pip install --upgrade openai langgraph langchain-core

PASTE ACCESS TOKEN FROM HF

In [3]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "paste access token here"

In [4]:
from openai import OpenAI
import os
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.environ["HUGGINGFACEHUB_API_TOKEN"],
)


In [5]:
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2:featherless-ai"

def llm_send(messages, model=MODEL_ID, temperature=0.7, max_tokens=512):
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    choice = completion.choices[0]
    content = None
    try:
        msg = choice.message
        if isinstance(msg, dict):
            content = msg.get("content") or msg.get("content", None)
        else:
            content = getattr(msg, "content", None)
            if isinstance(content, dict):
                content = content.get("text") or content.get("text", [None])[0] if content else None
    except Exception:
        pass
    if not content:
        content = getattr(choice, "text", None) or str(choice)

    return content


In [6]:
from typing import TypedDict, List, Dict
from langgraph.graph import StateGraph, START, END

class ChatState(TypedDict):
    messages: List[Dict[str, str]]

def hf_chat_node(state: ChatState) -> ChatState:
    messages = state.get("messages", [])
    assistant_text = llm_send(messages)
    messages = messages + [{"role": "assistant", "content": assistant_text}]
    return {"messages": messages}

graph = StateGraph(ChatState)
graph.add_node("chat", hf_chat_node)
graph.add_edge(START, "chat")
graph.add_edge("chat", END)
app = graph.compile()


In [8]:
initial_messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

state = {"messages": initial_messages.copy()}

print("LangGraph + HF Router Chat (type 'exit' to quit)\n")

while True:
    user = input("User: ").strip()
    if user.lower() in ("exit", "quit"):
        print("Goodbye.")
        break
    state["messages"].append({"role": "user", "content": user})
    out_state = app.invoke(state)
    assistant_reply = out_state["messages"][-1]["content"]
    print("Assistant:", assistant_reply, "\n")
    state = out_state


LangGraph + HF Router Chat (type 'exit' to quit)

User: hii my name is manshu
Assistant:  Hello Manshu, nice to meet you. How can I assist you today? If you have any questions or need help with a specific task, feel free to ask. 

User: can u tell me what is my name?
Assistant:  Manshu, you introduced yourself at the beginning of our conversation. So, your name is Manshu. Is there something specific you would like help with regarding your name or any other topic? Let me know how I can assist you. 

User: can u multiply 100 with 3
Assistant:   Yes, I can help you with that. The product of multiplying 100 by 3 is 300. So, 100 multiplied by 3 equals 300. Let me know if you have any other questions or need help with anything else. 

User: now divide the result by 2
Assistant:   To divide the result by 2, we can perform the following calculation:

300 ÷ 2 = 150

So, the result of multiplying 100 by 3 and then dividing by 2 is 150. Let me know if you have any other questions or need help wit